# Getting Started

In [1]:
%reload_ext autoreload
%autoreload 2
from recsys.MovieLens import MovieLens
from surprise import SVD

In [2]:
def create_anti_testset(user_id, trainset):
    fill = trainset.global_mean
    anti_testset = []
    inner_uid = trainset.to_inner_uid(str(user_id))
    
    user_items = set([j for (j, _) in trainset.ur[inner_uid]])
    anti_testset += [(trainset.to_raw_uid(inner_uid), trainset.to_raw_iid(i), fill) for
                             i in trainset.all_items() if
                             i not in user_items]
    return anti_testset

In [ ]:
# Pick an arbitrary test subject
test_uid = 85

lens, ratings_data, rankings = MovieLens.load()

user_ratings = lens.get_user_ratings(test_uid)
loved = []
hated = []
for ratings in user_ratings:
    if (float(ratings[1]) > 4.0):
        loved.append(ratings)
    if (float(ratings[1]) < 3.0):
        hated.append(ratings)

loved.sort(key=lambda x: x[1], reverse=True)
hated.sort(key=lambda x: x[1])

print(f"User {test_uid} loved these movies:")
for ratings in loved[:10]:
    print(lens.get_movie_name(ratings[0]), ratings[1])
print("\n...and hated these movies:")
for ratings in hated[:10]:
    print(lens.get_movie_name(ratings[0]), ratings[1])

In [ ]:
# Building recommendation model
trainset = ratings_data.build_full_trainset()

model = SVD(verbose=False)
model.fit(trainset)

In [ ]:
# Generate recommendations
testset = create_anti_testset(test_uid, trainset)
predictions = model.test(testset)

recommendations = []

ratings_data = lens.load_movielens_data()

print ("We recommend:\n")
for user_id, movie_id, true_rating, predicted_rating, _ in predictions:
    recommendations.append((int(movie_id), predicted_rating))

recommendations.sort(key=lambda x: x[1], reverse=True)

for ratings in recommendations[:10]:
    print(lens.get_movie_name(ratings[0]), f"{ratings[1]:.1f}")